In [22]:
from ctgan import load_demo
from ctgan import CTGAN
import pandas as pd

from sklearn.preprocessing import StandardScaler


In [23]:
df_m_0 = pd.read_csv('../dataset/min_0.csv')
df_m_1 = pd.read_csv('../dataset/min_1.csv')
df_m_2 = pd.read_csv('../dataset/min_2.csv')
df_m_3 = pd.read_csv('../dataset/min_3.csv')
df_0 = pd.read_csv('../dataset/informative_cluster_0.csv')
df_1 = pd.read_csv('../dataset/informative_cluster_1.csv')
df_2 = pd.read_csv('../dataset/informative_cluster_2.csv')

In [24]:
df_m_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4115 entries, 0 to 4114
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      4115 non-null   int64  
 1   type            4115 non-null   int64  
 2   oldbalanceOrg   4115 non-null   float64
 3   newbalanceOrig  4115 non-null   float64
 4   oldbalanceDest  4115 non-null   float64
dtypes: float64(3), int64(2)
memory usage: 160.9 KB


In [25]:
df = df_m_0

In [26]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,type,oldbalanceOrg,newbalanceOrig,oldbalanceDest
0,1,181.00,0.0,21182.0
1,1,2806.00,0.0,26202.0
2,1,20128.00,0.0,6268.0
3,1,0.00,0.0,102.0
4,1,1277212.77,0.0,0.0


In [27]:
# Names of the columns that are discrete
discrete_columns = [
    'type'
]

ctgan = CTGAN(epochs=10)
ctgan.fit(df, discrete_columns)

# Create synthetic data


In [28]:
synthetic_data = ctgan.sample(1000)

In [29]:
synthetic_data.head()

,type,oldbalanceOrg,newbalanceOrig,oldbalanceDest
0,1,-173522.783467,100.079221,-125908.788228
1,1,46875.634383,60.363389,976435.709896
2,1,190903.993423,159.793368,-214486.182078
3,1,-184829.164561,196.914430,-91139.431833
4,1,-177424.831260,-123.455997,-495523.518584


In [ ]:
from sdv.cag import CustomConstraint
from sdv.cag import SingleTableProgrammableConstraint

class TransactionBalanceLogic(CustomConstraint):
    def is_valid(self, row):
        t = row['type']
        obo = row['oldBalanceOrig']
        nbo = row['newBalanceOrig']
        obd = row['oldBalanceDest']

        if any(v is None for v in [t, obo, nbo, obd]):
            return True  # skip NaNs

        # Calculate transaction amount
        amount = obo - nbo

        if t == 'TRANSFER':
            return amount > 0 and obd + amount >= 0
        elif t == 'CASH_OUT':
            return amount > 0
        elif t == 'PAYMENT':
            return amount > 0
        elif t == 'DEBIT':
            return amount > 0 and obd + amount >= 0
        elif t == 'CASH_IN':
            return amount == 0 and obd > 0
        else:
            return True  # for unknown or test types

    def transform(self, table_data):
        return table_data  # no transformation needed

    def reverse_transform(self, table_data):
        return table_data


In [ ]:
from sdv.single_table import CTGANSynthesizer

synthesizer = CTGANSynthesizer(metadata)
synthesizer.add_constraints([TransactionBalanceLogic()])
synthesizer.fit(real_data)